In [1]:
import pyvisa as pv
import numpy as np
from math import log10
from pathlib import Path
import shutil
import time
from time import sleep
import matplotlib.pyplot as plt

# Initialize the Resource Manager
rm = pv.ResourceManager()

# List all connected VISA devices
visa_devices = rm.list_resources()
print("Connected Devices:")
for device in visa_devices:
    print(device)

Connected Devices:
USB0::0x0957::0x5707::MY59004733::INSTR
USB0::0x2A8D::0x1770::MY63420204::INSTR


In [2]:
# Instrument resource names (update as per your setup)
FUNC_GEN_ID = "USB0::0x0957::0x5707::MY59004733::INSTR"  
OSCILLOSCOPE_ID = "USB0::0x2A8D::0x1770::MY63420204::INSTR"  
# SOURCE_ID = "USB0::0x2A8D::0x1770::MY63420113::0::INSTR"  #voltage source


In [ ]:
# Enable coherent sampling 
from sympy import primerange, nextprime

def calculate_coherent_frequency(sample_rate, input_freq, points):
    bin_freq = round(sample_rate / points, 2)
    effective_freq = round(bin_freq * points, 2)
    num_cycles = input_freq / effective_freq * points
    sampling_window = round(num_cycles) + (1 if round(num_cycles) % 2 == 0 else 0)
    actual_freq = round(effective_freq * sampling_window / points, 2)
    return effective_freq, actual_freq

# Parameters
fsample = 1e6
start_freq = 1e3
stop_freq = 100e3
totalpoints = 2**13

# Frequency range setup
start = log10(start_freq)
stop = log10(stop_freq)
frequencies_SNR = np.logspace(start, stop, num=20, endpoint=True, base=10.0)   #generate all frequencies for the sweep 
frequencies_SNR = np.insert(frequencies_SNR, 0, 2e3)   # add 2k as the 1. freq
frequencies_SNR = np.append(frequencies_SNR, 250e3)    # add 250k as the last freq

# Initialize arrays
clock_values = np.zeros(len(frequencies_SNR))
adjusted_frequencies = np.zeros(len(frequencies_SNR))

# Compute coherent sampling frequencies
for idx, freq in enumerate(frequencies_SNR):
    optimized_clock, coherent_frequency = calculate_coherent_frequency(fsample, freq, totalpoints)
    adjusted_frequencies[idx] = coherent_frequency
    clock_values[idx] = optimized_clock

In [ ]:
# Waveform generator function definitions

def INSconnect(instrument_id):   # Connect to every instrument

    func_gen = rm.open_resource(instrument_id)    # rm: Resource manager handle
    func_gen.write('*IDN?')      #ask for response
    resp = func_gen.read()       #read response from device
    print("Connected to:", resp)     #print out response

    return func_gen    # Handle of function generator, use this handle to make further controls

def AFGsetup(func_gen, channel, minV, maxV):

    max_voltage = 'SOURce'+str(channel)+':VOLTage:LIMit:HIGH ' + str(maxV)+'V'
    min_voltage = 'SOURce'+str(channel)+':VOLTage:LIMit:LOW ' + str(minV)+'V'
    limit_en = 'SOURce'+str(channel)+':VOLTage:LIMit:STATe 1'  #set state to 0 to disable voltage limit
    func_gen.write(max_voltage)
    func_gen.write(min_voltage)
    # func_gen.write(limit_en)
    print("Function generator set up done!")

    return

def AFGsinusoid(func_gen, channel, offset, amp, frequency, phase=0):    # Output a sinusoid from the function generator 
    #example: SOURce1:VOLTage:LEVel:IMMediate:OFFSet 500mV
    
    # SOURce1:FUNCtion SQU
    sin_shape = 'SOURce' + str(channel) +':FUNCtion SIN'  #output shape
    func_gen.write(sin_shape)

    sin_phase = 'SOURce' + str(channel) + ':BURSt:PHASe ' + str(phase) #phase of the signal = 0
    func_gen.write(sin_phase)

    # sin_mode = 'SOURce' + str(channel) + ':FREQuency:MODE FIXed'    # set freq mode, other modes are CW/ FIXed/ SWEep
    # func_gen.write(sin_mode)

    # SOURce1:FREQuency +1.0E+06
    sin_frequency = 'SOURce' + str(channel) + ':FREQuency:FIXed ' + str(frequency)    # set freq
    func_gen.write(sin_frequency)

    # SOURce1:VOLTage:OFFSet +0.5
    sin_offset = 'SOURce' + str(channel) + ':VOLTage:LEVel:IMMediate:OFFSet ' + str(offset) # set offset
    func_gen.write(sin_offset)

    # VOLT 5 Vpp
    sin_unit = 'SOURce' + str(channel) + ':VOLTage:UNIT VPP' #set unit to voltage peak to peak (VPP)
    func_gen.write(sin_unit)                               #or set them as VRMS, DBM, VDC

    # SOURce1:VOLTage +0.5
    sin_amplitude = 'SOURce' + str(channel) + ':VOLTage:LEVel:IMMediate:AMPLitude ' + str(amp)
    func_gen.write(sin_amplitude)

    # Enable High-Z mode
    highz_command = 'OUTPut' + str(channel) + ':LOAD INF'
    func_gen.write(highz_command)
    
    print(f"Set sinusoid with {frequency} Hz, {amp}V amplitude, {offset}V offset")

    return

def output_en(func_gen,channel):   # Enable Output at channel 1|2
    output_enable = 'OUTPut' + str(channel) + ':STATE ON'
    func_gen.write(output_enable)
    return

def output_dis(func_gen,channel):   # Disable Output at channel 1|2
    output_disable = 'OUTPut' + str(channel) + ':STATE OFF'
    func_gen.write(output_disable)
    return


def AFGsquare(func_gen, channel, offset, amp, frequency, duty_cycle=50, phase=0): 
    # Set wave shape to square
    func_gen.write(f'SOURce{channel}:FUNCtion SQU')

    # Set phase
    func_gen.write(f'SOURce{channel}:BURSt:PHASe {phase}')

    # Set frequency
    func_gen.write(f'SOURce{channel}:FREQuency:FIXed {frequency}')

    # Set offset
    func_gen.write(f'SOURce{channel}:VOLTage:LEVel:IMMediate:OFFSet {offset}')

    # Set amplitude
    func_gen.write(f'SOURce{channel}:VOLTage:LEVel:IMMediate:AMPLitude {amp}')

    # Set duty cycle
    func_gen.write(f'SOURce{channel}:PULSe:DCYCle {duty_cycle}')
    
    print(f"Set square wave with {frequency} Hz, {amp}V amplitude, {offset}V offset, {duty_cycle}% duty cycle")

    return

def Phase_syn(func_gen):
    func_gen.write(":SOURce:PHASe:SYNChronize")
    return

In [ ]:
# Oscilloscope function definitions

def OSCIsetup(oscilloscope):

    # Enable Bus 0-8 for digital bits and 10 for EOC
    oscilloscope.write(":BUS1:BITS (@0:15),0")
    oscilloscope.write(":BUS1:BITS (@0:8,10),1")

    oscilloscope.write(":RUN")

    oscilloscope.write(':DIG:BUS1:DISP ON')  # Enable digital bus display
    print('Digital channels D0-D9 & D10 enabled.')

    # set up threshold voltage
    for i in range(16):
        thr = ":DIGital"+str(i)+":THReshold 1.8"
        oscilloscope.write(thr)

    # set up trigger digital channel, in our case EOC
    oscilloscope.write(":TRIGger:MODE EDGE")
    oscilloscope.write(":TRIGger:EDGE:SLOPe POSitive")
    oscilloscope.write(":TRIGger:EDGE:SOURce DIGital 10")

    # define timebase
    oscilloscope.write(":TIMebase:SCALe 1/1e3")

    print("Oscilloscope setup complete.")
    
    return

def save_d_data(data, freq):
    # Generate a dynamic filename based on frequency
    filename = f"digital_data_{freq:.2f}Hz.txt"

    # Open the file in write mode and save the data
    with open(filename, 'w') as file:
        file.write(f"{data}\n")
    print(f"[DEBUG] Digital data saved to {filename}.")
    return

def fetch_waveform_data(oscilloscope):     #Configure the oscilloscope to acquire waveform data and fetch it.
    # Configure the oscilloscope for digital bus acquisition
    oscilloscope.write(":WAVEFORM:SOURCE BUS1")  # Set waveform source to BUS1
    oscilloscope.write("WAVEFORM:BYTEORDER MSBFirst")  # Set byte order to MSB first
    oscilloscope.write(":WAVEFORM:FORMAT ASCII")  # Set waveform format to ASCII
    oscilloscope.write("WAVEFORM:POINTS MAX")  # Set waveform points to maximum

    # Perform a single acquisition
    oscilloscope.write(":SINGLE")

    # Fetch the waveform data
    data = oscilloscope.query(":WAVEFORM:DATA?")
    preamble = scope.query(":WAVEFORM:PREAMBLE?")
    
    return  preamble, data


In [18]:
# Connection attempt

func_gen = INSconnect(FUNC_GEN_ID)
oscilloscope = INSconnect(OSCILLOSCOPE_ID)
# power = INSconnect(SOURCE_ID)

Connected to: Agilent Technologies,33622A,MY59004733,A.02.03-3.15-03-64-02

Connected to: KEYSIGHT TECHNOLOGIES,MSO-X 3104G,MY63420204,07.60.2023080430



In [19]:
# Do frequency seweep and gather all data at once

# Prepare the waveform generator
output_en(func_gen, channel=1)
output_en(func_gen, channel=2)
AFGsquare(func_gen, channel=2, offset=1.25, amp=2.5, frequency=1.0E+6, phase=0)
sin_amp = 1

# Frequencies sweep from coherent frequencies
for index, sin_freq in enumerate(adjusted_frequencies):
    # Sin Input parameters
    AFGsinusoid(func_gen, channel=1, offset=0, sin_amp=sin_amp, frequency=sin_freq, phase=0)  # Setup sine wave
    FG.Sync_phase(fg)  # Synchronize the phase
    print(f"frequency @{sin_freq} Hz")  # Print current frequency

    # Prepare the oscilloscope
    OSCIsetup(oscilloscope)

    # Fetch data
    [preamble, data] = fetch_waveform_data(oscilloscope)
    combined_string = ''.join(data)
    preamble_string = ''.join(preamble)
    # Split them by comma
    rows = combined_string.split(',')
    row_preamble = preamble_string.split(',')
    new_data = []
    i = 0
    for raw in rows[1:]:
        new_data.append(int(raw, 16))
        i = i + 1

    # Change the path if measuring another daughterboard!!
    PATH = r"C:\Users\Kai\Downloads\Lab3\Board3"
    folder_properties = "Freq_1k_250k-SIN_" + str(int(sin_amp)) + "Vpp-CLK_" + str(int(clk_freq) / 1e6) + "MHz"
    folder_name = "Kai-Board3" + folder_properties
    file_path = os.path.join(PATH, folder_name)
    os.makedirs(file_path, exist_ok=True)

    # Create the file
    filename = "SIN_" + str(sin_freq) + "Hz_" + ".csv"
    file_path = os.path.join(file_path, filename)

    # Write the data into csv file
    with open(file_path, mode='w', newline='') as output_file:
        csv_writer = csv.writer(output_file)
        # Save each number as a new row
        for value in new_data:
            csv_writer.writerow([value])
    print(f"Data successfully written to {file_path}")
    
    # Close the connection (at the end of the loop)
    output_dis(func_gen, channel=1)
    output_dis(func_gen, channel=2)
    oscilloscope.write(":RUN") 
    sleep(3)


Set sinusoid with 2000.0 Hz, 1V amplitude, 0V offset
Set square wave with 1000000.0 Hz, 2.5V amplitude, 1.25V offset, 50% duty cycle
Digital channels D0-D9 enabled.
Binary data saved as signal_freq_2000.bin
